In [2]:
import importlib.util

package_name = "albumentations"
spec = importlib.util.find_spec(package_name)

if spec is not None:
    print(f"{package_name} is installed!")
else:
    print(f"{package_name} is NOT installed!")


albumentations is installed!


In [4]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

# Define input and output directories
input_dirs = ["benign", "malignant"]  # Folders where original images are stored
output_parent_dir = "augmented_images"  # Main folder to store augmented images

# Define augmentation techniques
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])

# Number of augmented copies per image
num_augmented_copies = 3  # Change this to generate more or fewer images

# Function to augment and save images
def augment_and_save_images(class_name):
    input_folder = class_name  # Since folders are in the main working directory
    output_folder = os.path.join(output_parent_dir, class_name)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Process each image
    for img_name in tqdm(os.listdir(input_folder), desc=f"Augmenting {class_name}"):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)
        
        if image is None:
            continue  # Skip unreadable files
        
        # Convert image from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        for i in range(num_augmented_copies):  # Generate multiple augmented copies
            augmented = augmentations(image=image)['image']
            
            # Convert back to BGR for saving with OpenCV
            augmented = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
            
            # Save augmented image with "aug_" prefix and unique index
            aug_img_name = f"aug_{i}_{img_name}"
            aug_img_path = os.path.join(output_folder, aug_img_name)
            cv2.imwrite(aug_img_path, augmented)

# Perform augmentation for both classes
for class_name in input_dirs:
    augment_and_save_images(class_name)

print("Augmentation completed successfully!")


Augmenting malignant: 100%|██████████████████████████████████████████████████████████| 288/288 [00:07<00:00, 36.94it/s]

Augmentation completed successfully!
